In [ ]:
!pip install pandas_datareader

In [ ]:
!pip install yfinance

In [ ]:
!pip install html5lib

In [9]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from unidecode import unidecode
import re
import os

## Buscas empresas listadas no IBOVESPA

In [ ]:
url_ibovespa = 'https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-amplos/indice-ibovespa-ibovespa-composicao-da-carteira.htm'

response = requests.get(url_ibovespa)
html = response.content
soup = BeautifulSoup(html)

#tabelas_b3 = pd.read_html(url_ibovespa)

#url_ibovespa
soup.prettify()

In [ ]:
table = soup.find_all('table')
table

In [33]:
def create_df(index_name):
    '''Essa é uma função que recebe o nome do índice da Bolsa de Valores e retorna um DataFrame com todas as empresas e seus respectivos setores'''

    #criando DataFrame
    index_name = pd.read_csv('data/'+str(index_name)+'_setor.csv', 
                                encoding='ISO-8859-1',
                                header=1,
                                sep=';',
                                decimal=',',
                                thousands='.',
                                skipfooter=2,
                                index_col=False)
    
    #normalizando colunas
    index_name.columns = [re.sub('[\.()]', '', re.sub(' ', '_', unidecode(columns.lower()))) for columns in index_name.columns]

    return index_name

## Criando DataFrame com empresas da Ibovespa

In [ ]:
ibov = create_df('ibov')
ibov

In [ ]:
ibov = pd.read_csv('data/ibov_setor.csv', 
                                encoding='ISO-8859-1',
                                header=1,
                                sep=';',
                                decimal=',',
                                thousands='.',
                                skipfooter=2,
                                index_col=False)

## Criando DataFrame com empresas do IBRX100

In [10]:
ibrx = create_df('ibrx100')

C:\Users\Pedro\AppData\Local\Temp\ipykernel_11816\1620139615.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  index_name = pd.read_csv('data/'+str(index_name)+'_setor.csv',


## Verificando empresas que divergem nos dois índices

In [24]:
emp_ibrx = ibrx['codigo']
set_ibrx = set(emp_ibrx)
len(set_ibrx)

100

In [25]:
emp_ibov = ibov['codigo']
set_ibov = set(emp_ibov)
len(set_ibov)

92

In [31]:
print(len(set_ibov.difference(set_ibrx)))
print(set_ibov.difference(set_ibrx))

1
{'POSI3'}


In [32]:
print(len(set_ibrx.difference(set_ibov)))
print(set_ibrx.difference(set_ibov))

9
{'MOVI3', 'VAMO3', 'STBP3', 'MDIA3', 'AURE3', 'PSSA3', 'ENAT3', 'AMBP3', 'CBAV3'}


In [34]:
pd.concat([ibrx, ibov]).drop_duplicates()

,setor,codigo,acao,tipo,qtde_teorica,part_%,part_%acum
0,Bens Indls / Máqs e Equips,WEGE3,WEG,ON NM,1481593024,2.718,2.718
1,Bens Indls / Mat Transporte,EMBR3,EMBRAER,ON NM,734632205,0.433,0.433
2,Bens Indls/Transporte,AZUL4,AZUL,PN N2,327646296,0.205,2.331
3,Bens Indls/Transporte,CCRO3,CCR SA,ON NM,1115681435,0.579,2.331
4,Bens Indls/Transporte,ECOR3,ECORODOVIAS,ON NM,333657615,0.072,2.331
...,...,...,...,...,...,...,...
87,Utilidade Públ / Energ Elétrica,ENGI11,ENERGISA,UNT N2,248477689,0.542,9.458
88,Utilidade Públ / Energ Elétrica,ENEV3,ENEVA,ON NM,1561135009,0.957,9.458
89,Utilidade Públ / Energ Elétrica,EGIE3,ENGIE BRASIL,ON NM,255208461,0.494,9.458
90,Utilidade Públ / Energ Elétrica,EQTL3,EQUATORIAL,ON NM,1100513485,1.478,9.458
